<a href="https://colab.research.google.com/github/svplaksin/simulative-python/blob/main/%D0%A2%D1%80%D0%B5%D1%82%D0%B8%D0%B9_%D0%BA%D0%B5%D0%B9%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3 кейс

**В этом кейсе вы будете рассчитывать:**
* retention
* rolling retention
* lifetime
* churn rate
* mau
* wau
* dau

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [1]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

--2025-08-20 13:12:39--  https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.github.com (gist.github.com)... 140.82.113.4
Connecting to gist.github.com (gist.github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv [following]
--2025-08-20 13:12:39--  https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14918 (15K) [text/plain]
Saving to: ‘registrations.csv.1’

registrations.csv.1 100%[===================>]  14.57K  --.-KB/s    in 0.001s  

2025-08-20 13:12:39 (11.7

Файлами для работы являются `registrations.csv` и `entries.csv`. В них хранятся данные о регистрациях пользователей и входа на платформу соответственно.

### **Посчитайте Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе**

Cохраните результат в переменную `retention_15_day`

**Примечание:** результат округлите до 5 знаков после запятой

In [2]:
import csv
from datetime import datetime, timedelta
from statistics import fmean

In [3]:
def parse_date(date_str):
    # Парсим дату из строки в объект datetime
    return datetime.strptime(date_str, '%Y-%m-%d')

def calculate_retention(reg_file, login_file):
    # Словарь для хранения дат регистрации пользователей
    user_registrations = {}

    # 1. Чтение файла с регистрациями
    with open(reg_file, mode='r') as f:
        reader = csv.reader(f, delimiter=';')
        next(reader)  # пропускаем заголовок, если есть

        for row in reader:
            user_id, reg_date_str = row
            reg_date = parse_date(reg_date_str)

            # Отбираем  январских пользователей
            if reg_date.month == 1:
                user_registrations[user_id] = reg_date

    # 2. Чтение файла с входами и подсчет пользователей, вошедших через 15 дней после регистрации
    retained_users = set()

    with open(login_file, mode='r') as f:
        reader = csv.reader(f, delimiter=';')
        next(reader)  # пропускаем заголовок, если есть

        for row in reader:
            user_id, login_date_str = row

            # Проверяем, есть ли пользователь в нашем словаре регистраций
            if user_id in user_registrations:
                reg_date = user_registrations[user_id]
                login_date = parse_date(login_date_str)

                # Вычисляем разницу в днях между входом и регистрацией
                delta = (login_date - reg_date).days

                # Если пользователь вошел на 15 день после регистрации, добавляем его
                if delta == 15:
                    retained_users.add(user_id)

    # 3. Расчет retention
    total_january_users = len(user_registrations)
    retained_count = len(retained_users)

    if total_january_users == 0:
        return 0.0  # чтобы избежать деления на ноль

    retention_rate = round((retained_count / total_january_users) * 100, 5)
    return retention_rate


reg_file = 'registrations.csv'
login_file = 'entries.csv'
retention_15_day = calculate_retention(reg_file, login_file)
print(f"Retention 15 дня для январских пользователей: {retention_15_day}%")

Retention 15 дня для январских пользователей: 54.65116%


In [4]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
# Открываем файл с правильными ответами
with open('metrics.txt', 'r') as f:
    answers = f.read().split('\n')

correct_answer = float(answers[0])

try:
    assert retention_15_day == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Rolling-retention 30 дня (в процентах) для пользователей из той же когорты**

Сохраните результат в переменную `rolling_retention`

**Примечание:** результат округлите до 5 знаков после запятой

In [5]:
import csv
from datetime import datetime, timedelta

def parse_date(date_str):
    # Парсим дату из строки в объект datetime
    return datetime.strptime(date_str, '%Y-%m-%d')

def calculate_roll_retention(reg_file, login_file, n_day = 30, month = None):
    # Словарь для хранения дат регистрации пользователей
    user_registrations = {}

    # 1. Чтение файла с регистрациями
    with open(reg_file, mode='r') as f:
        reader = csv.reader(f, delimiter=';')
        next(reader)  # пропускаем заголовок, если есть

        for row in reader:
            user_id, reg_date_str = row
            reg_date = parse_date(reg_date_str)

            # Отбираем  январских пользователей
            if month:
              if reg_date.month == month:
                user_registrations[user_id] = reg_date
            else:
              user_registrations[user_id] = reg_date

    # 2. Чтение файла с входами и подсчет пользователей, вошедших через 15 дней после регистрации
    retained_users = set()

    with open(login_file, mode='r') as f:
        reader = csv.reader(f, delimiter=';')
        next(reader)  # пропускаем заголовок, если есть

        for row in reader:
            user_id, login_date_str = row

            # Проверяем, есть ли пользователь в словаре регистраций
            if user_id in user_registrations:
                reg_date = user_registrations[user_id]
                login_date = parse_date(login_date_str)

                # Вычисляем разницу в днях между входом и регистрацией
                delta = (login_date - reg_date).days

                # Если пользователь вошел на 30 день после регистрации, или позднее, добавляем его
                if delta >= n_day:
                    retained_users.add(user_id)

    # 3. Расчет retention
    total_january_users = len(user_registrations)
    retained_count = len(retained_users)

    if total_january_users == 0:
        return 0.0  # чтобы избежать деления на ноль

    rolling_retention = round((retained_count / total_january_users) * 100, 5)
    return rolling_retention


reg_file = 'registrations.csv'
login_file = 'entries.csv'
rolling_retention = calculate_roll_retention(reg_file, login_file, n_day=30, month=1)
print(f"Rolling Retention 30 дня для январских пользователей: {rolling_retention}%")


Rolling Retention 30 дня для январских пользователей: 29.06977%


In [6]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[1])

try:
    assert rolling_retention == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Lifetime по всем пользователям, посчитанный как интеграл от n-day retention**

Сохраните результат в переменную `lifetime`

**Примечание:** результат округлите до 5 знаков после запятой

In [7]:
from collections import defaultdict

def calculate_ltv(reg_file, login_file):
  user_registrations = {}

  with open(reg_file, 'r') as f:
    reader = csv.reader(f, delimiter=';')
    next(reader)

    for row in reader:
      user_id, reg_date_str = row
      reg_date = parse_date(reg_date_str)
      user_registrations[user_id] = reg_date

  # Находим для  каждого юзера все дни активности
  user_active_days = defaultdict(set)

  with open(login_file, 'r') as f:
    reader = csv.reader(f, delimiter=';')
    next(reader)
    for row in reader:
      user_id, login_date_str = row
      if user_id in user_registrations:
        reg_date = user_registrations[user_id]
        login_date = parse_date(login_date_str)
        delta_days = (login_date - reg_date).days
        if delta_days >= 0:
          user_active_days[user_id].add(delta_days)
  # Считаем LTV
  total_users = len(user_registrations)
  retention_by_day = defaultdict(int)

  for user_id, active_days in user_active_days.items():
    for day in active_days:
      retention_by_day[day] += 1
  # Retention для каждого дня в долях
  retention_rates = {day: round(count / total_users, 5) for day, count in retention_by_day.items()}
  lifetime = round(sum(retention_rates.values()),5)
  return lifetime


lifetime = calculate_ltv(reg_file, login_file)

In [8]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[2])

try:
    assert lifetime == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Churn rate 29 дня (в долях), посчитанный по всем пользователям**

Сохраните результат в переменную `churn_29`.  
Если будете считать CR от Retention, ведите расчет от Rolling Retention, таким образом, мы получим всех, кто не заходил в 29 день и после. Ведя расчет от обычного Retention, наоборот - получим только CR в 29 день.


In [9]:
churn_29 = 1 - calculate_roll_retention(reg_file, login_file, 29) / 100

In [10]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[3])

try:
    assert churn_29 == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau за последний месяц/неделю/день записей**

Сохраните результат в переменные `dec_mau`, `dec_wau`, `dec_dau` соответственно

**Примечание:** последний месяц записей - декабрь. Поэтому `mau` рассчитываем для декабря (2021 года), для `wau` берем последнюю неделю - с 25 по 31 декабря, и для `dau` соответственно последний день - 31 декабря.

In [11]:
def calculate_active_users(login_file, month=False, week=False, day=False):
  with open(login_file, 'r') as f:
    reader = csv.reader(f, delimiter=';')
    next(reader)

    active_users = set()
    for row in reader:
      user_id, date_str = row
      date = parse_date(date_str)
      if month:
        if date.month == month:
          active_users.add(user_id)
      if week:
        if date >= datetime(2021, 12, 25):
          active_users.add(user_id)
      if day:
        if date >= datetime(2021, 12, 31):
          active_users.add(user_id)
    return len(active_users)

dec_mau = calculate_active_users(login_file, month=12)
dec_mau

133

In [12]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[4])

try:
    assert dec_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [13]:
dec_wau = calculate_active_users(login_file, week=True)
dec_wau

84

In [14]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[5])

try:
    assert dec_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [15]:
dec_dau = calculate_active_users(login_file, day=True)
dec_dau

47

In [16]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[6])

try:
    assert dec_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau усредненные**

Сохраните результат в переменные `avg_mau`, `avg_wau`, `avg_dau` соответственно

**Примечание:** результаты округлите до 5 знаков после запятой

In [17]:
def calculate_active_users(login_file, month=None, week=False, day=False, average = False):
  with open(login_file, 'r') as f:
    reader = csv.reader(f, delimiter=';')
    next(reader)

    active_users = {m: set() for m in range(1, 13)}
    for row in reader:
      user_id, date_str = row
      date = parse_date(date_str)
      if month:
        active_users[date.month].add(user_id)
    active_users = {x: len(y) for x,y in active_users.items()}
    average = round(fmean(active_users.values()), 5)
    return average


avg_mau = calculate_active_users(login_file, month=True, average=True)
avg_mau

102.58333

In [18]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[7])

try:
    assert avg_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [19]:
from statistics import fmean
def calculate_avg_wau(login_file):
  with open(login_file, 'r') as f:
    reader = csv.reader(f, delimiter=';')
    next(reader)

    active_users = {m: set() for m in range(1, 54)}
    for row in reader:
      user_id, date_str = row
      week_number = parse_date(date_str).isocalendar().week
      active_users[week_number].add(user_id)
    active_users = {x: len(y) for x,y in active_users.items()}
    average = round(fmean(active_users.values()), 5)
    return average


avg_wau = calculate_avg_wau(login_file)

avg_wau

89.86792

In [20]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[8])

try:
    assert avg_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [21]:
from statistics import fmean
def calculate_avg_dau(login_file):
  with open(login_file, 'r') as f:
    reader = csv.reader(f, delimiter=';')
    next(reader)

    active_users = {m: set() for m in range(1, 366)}
    for row in reader:
      user_id, date_str = row
      day_number = parse_date(date_str).timetuple().tm_yday
      active_users[day_number].add(user_id)
    active_users = {x: len(y) for x,y in active_users.items()}
    average = round(fmean(active_users.values()), 5)
    return average


avg_dau = calculate_avg_dau(login_file)

avg_dau

40.5589

In [22]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[9])

try:
    assert avg_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
